In [1]:
# Some useful Google API documentation:
# https://developers.google.com/maps/documentation/directions/
# https://developers.google.com/maps/documentation/roads/snap
from modules.API_Keys import StreetView_API_Key
from modules.utils import haversine, get_sv_img, get_video, calculate_angle
import os
import json
import pandas as pd
import cv2
import time
import sys
import requests
import math
apikey_streetview = StreetView_API_Key()
metadata=[]

In [2]:
def get_sv_images(apikey_streetview,
                  start_coords,
                  end_coords,
                  savepath='PHOTO_FOLDER/',
                  pic_dim="640x450"):
    
    distance = haversine(start_coords, end_coords)
    num_images = max(1, int(distance // 30))  # At least 1 image for every 3 feet

    lat_step = (end_coords[0] - start_coords[0]) / num_images
    lon_step = (end_coords[1] - start_coords[1]) / num_images

    global metadata
    direction_angle = calculate_angle(start_coords,end_coords)

    for i in range(num_images):
        lat = start_coords[0] + lat_step * i
        lon = start_coords[1] + lon_step * i
        timestamp = time.time_ns()
        filename = f"image_{timestamp}"
        
        get_sv_img(apikey_streetview, (lat, lon), filename=filename, savepath=savepath, picsize=pic_dim, heading=direction_angle)
        
        metadata.append({
            "image_name": filename + ".jpg",
            "latitude": lat,
            "longitude": lon,
            "image_dim": pic_dim
        })
        
    return

In [3]:
images_path='Data'+os.sep+'PHOTO_FOLDER/'
coordiantes_csv ='Coordinates.csv'
pic_dim = "640x640"  # Adjusted for efficiency
output_vid_fps=10

timestamp = time.time()
video_name = os.path.join('', f'SV_video_{timestamp}.mp4')

lat_long_points=pd.read_csv(coordiantes_csv)
total_segments=len(lat_long_points["Latitude_A"])

# Start Points
lat_points_A=lat_long_points["Latitude_A"]
long_points_A=lat_long_points["Longitude_A"]

# End Points
lat_points_B=lat_long_points["Latitude_B"]
long_points_B=lat_long_points["Longitude_B"]

for idx in range(total_segments):
    start_point = (lat_points_A[idx], long_points_A[idx])
    end_point= (lat_points_B[idx], long_points_B[idx])
    print(f"Processing segment {idx + 1}: Start {start_point}, End {end_point}")
    get_sv_images(apikey_streetview, start_point, end_point, savepath=images_path)

# Save metadata
metadata_df = pd.DataFrame(metadata)
metadata_df.to_csv((str(timestamp)+'metadata.csv'), index=False)
get_video(images_path,video_name,output_vid_fps)
metadata=[]

Processing segment 1: Start (25.09639043, 55.15337034), End (25.09854667, 55.15531477)
